In [1]:
include("quantum.jl")

matrixplot (generic function with 2 methods)

# Single Qubit Reconstruction using SIC-POVM

In [3]:
# define sic povm for d=2
ψ₀ = Ket([1,0])
ψ₁ = Ket([1/sqrt(3), sqrt(2/3)])
ψ₂ = 1/sqrt(3) * Ket([1, sqrt(2)*exp(2*pi*im/3)])
ψ₃ = 1/sqrt(3) * Ket([1, sqrt(2)*exp(4*pi*im/3)])

# create vector of projectors
ψs = [ψ₀, ψ₁, ψ₂, ψ₃]

# convert to operators
π̂s = map(density, ψs);

In [4]:
# create a random state
ϕ₂ = rand(Ket{2})

# perform measurements on random state
meaurements_ϕ₂ = 1/2 * evaluate.([measurement(ϕ₂, π̂) for π̂ in π̂s])

# reconstruct random state from meausrement results
ϕ₂_reconstructed = 3 * sum([measurement*π̂ for (measurement, π̂) in zip(meaurements_ϕ₂, π̂s)]) - Î(2)

# measure the fidelity of the reconstructed state
fidelity_ϕ₂ = fidelity(density(ϕ₂), ϕ₂_reconstructed)

1.0000000000000002

# Two Qubit Reconstruction using SIC-POVM

In [4]:
# create a random state
ϕ₄ = rand(Ket{4})

# perform measurements on random state
measurements_ϕ₄ = 1/4 * evaluate.([measurement(ϕ₄, sicpovm_matrix) for sicpovm_matrix in sicpovm_matrices])

# reconstruct random state from measurement results
ϕ₄_reconstructed = 5 * sum([measurement*sicpovm_matrix for (measurement, sicpovm_matrix) in zip(measurements_ϕ₄, sicpovm_matrices)]) - Î(4)

# meausre the fidelity of the reconstructed state
fidelity_ϕ₄ = fidelity(density(ϕ₄), ϕ₄_reconstructed)

0.9999999999999999

# Two Qubit Reconstruction using SIC-POVM Tensor Product

In [5]:
# define tensor product SIC-POVM
Π̂s = 1/4 * [kron(π̂ᵢ, π̂ⱼ) for π̂ᵢ in π̂s for π̂ⱼ in π̂s]

# check that it sums to identity
@assert isapprox(sum(Π̂s), Î(4)) "Not a POVM"

# check that it is informationally complete (rank = 16)
@assert rank(hcat([reshape(adjoint(Π̂), 16) for Π̂ in data.(Π̂s)]...)) == 16 "Not informationally complete"

# check if all operators are self adjoint
@assert all(Π̂ -> isapprox(Π̂, dag(Π̂)), Π̂s) "Not self adjoint"